# Building a Practical LLM-Powered Solution

We are going to build a useful application powered by an OpenAI language model.

### The goal is to design a tool that acts like a smart web browser:

you provide it with a URL, and it summarizes the content for you in a concise and clear way—a true Reader's Digest for the Internet!

In [1]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

# Connecting to OpenAI

The next cell is where we load in the environment variables in our `.env` file and connect to OpenAI.

In [2]:
# Load environment variables in a file called .env

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


In [3]:
openai = OpenAI()

In [4]:
# A class to represent a Webpage

class Website:
    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [5]:
# Let's try one out. Change the website and add print statements to follow along.

website = Website("https://www.arshilegorkyfoundation.org/")
print(website.title)
print(website.text)

Home - Arshile Gorky Foundation
Skip to content
The Arshile Gorky Foundation
Foundation
Artist
Catalogue Raisonné
News
Menu
Foundation
Artist
Catalogue Raisonné
News
Arshile Gorky and his oldest daughter Maro moving from Roxbury to Sherman, Connecticut, September 1945. Photograph by Agnes Magruder Gorky.
View More
News
Arshile Gorky: New York City
Hauser & Wirth, New York
September 4–October 26, 2024
Hauser & Wirth
, New York,
Arshile Gorky: New York City
, September 4–October 26, 2024.
Arshile Gorky Catalogue Raisonné
November 12, 2021
The Arshile Gorky Foundation publishes the
Arshile Gorky Catalogue Raisonné
.
Arshile Gorky: Beyond The Limit
Hauser & Wirth, New York
November 16–December 23, 2021
Hauser & Wirth
, New York,
Arshile Gorky: Beyond
The Limit, November 16–December 23, 2021.
All works by Arshile Gorky © 2020 The Arshile Gorky Foundation / Artists Rights Society (ARS), New York.
Site Index
Back To Top


## Types of prompts

Models like GPT4o have been trained to receive instructions in a particular way.

They expect to receive:

**A system prompt** that tells them what task they are performing and what tone they should use

**A user prompt** the conversation starter that they should reply to

In [6]:
# Define our system prompt - you can experiment with this, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [7]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [8]:
print(user_prompt_for(website))

You are looking at a website titled Home - Arshile Gorky Foundation
The contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.

Skip to content
The Arshile Gorky Foundation
Foundation
Artist
Catalogue Raisonné
News
Menu
Foundation
Artist
Catalogue Raisonné
News
Arshile Gorky and his oldest daughter Maro moving from Roxbury to Sherman, Connecticut, September 1945. Photograph by Agnes Magruder Gorky.
View More
News
Arshile Gorky: New York City
Hauser & Wirth, New York
September 4–October 26, 2024
Hauser & Wirth
, New York,
Arshile Gorky: New York City
, September 4–October 26, 2024.
Arshile Gorky Catalogue Raisonné
November 12, 2021
The Arshile Gorky Foundation publishes the
Arshile Gorky Catalogue Raisonné
.
Arshile Gorky: Beyond The Limit
Hauser & Wirth, New York
November 16–December 23, 2021
Hauser & Wirth
, New York,
Arshile Gorky: Beyond
The Limit, November 16–December 23, 20

## Messages

The API from OpenAI expects to receive messages in a particular structure.
Many of the other APIs share this structure:

In [9]:
# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [10]:
# Try this out, and then try for a few more websites

messages_for(website)

[{'role': 'system',
  'content': 'You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. Respond in markdown.'},
 {'role': 'user',
  'content': 'You are looking at a website titled Home - Arshile Gorky Foundation\nThe contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.\n\nSkip to content\nThe Arshile Gorky Foundation\nFoundation\nArtist\nCatalogue Raisonné\nNews\nMenu\nFoundation\nArtist\nCatalogue Raisonné\nNews\nArshile Gorky and his oldest daughter Maro moving from Roxbury to Sherman, Connecticut, September 1945. Photograph by Agnes Magruder Gorky.\nView More\nNews\nArshile Gorky: New York City\nHauser & Wirth, New York\nSeptember 4–October 26, 2024\nHauser & Wirth\n,\xa0New York,\nArshile Gorky: New York City\n,\xa0September 4–October 26, 2024.\nArshile Gorky Catalogue Raisonné\nNovember 1

## Time to bring it together!

In [11]:
# And now: call the OpenAI API.

def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [12]:
summarize("https://www.arshilegorkyfoundation.org/")

"# Summary of the Arshile Gorky Foundation Website\n\nThe Arshile Gorky Foundation is dedicated to the life and works of the Armenian-American artist Arshile Gorky. It features information about the foundation's initiatives, Gorky's artistic contributions, and a comprehensive catalogue raisonné of his works.\n\n## News and Announcements\n\n- **Exhibition: Arshile Gorky: New York City**\n  - **Location**: Hauser & Wirth, New York\n  - **Dates**: September 4 – October 26, 2024\n\n- **Publication**: The Arshile Gorky Catalogue Raisonné was published on November 12, 2021.\n\n- **Exhibition: Arshile Gorky: Beyond The Limit**\n  - **Location**: Hauser & Wirth, New York\n  - **Dates**: November 16 – December 23, 2021\n\nThe foundation aims to promote Gorky's legacy while providing resources and exhibitions related to his work."

In [13]:
# A function to display this nicely in the Jupyter output, using markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [14]:
display_summary("https://www.arshilegorkyfoundation.org/")

# Summary of the Arshile Gorky Foundation Website

The Arshile Gorky Foundation is dedicated to the life and work of the influential artist Arshile Gorky. The website includes sections about the foundation and Gorky as an artist, as well as information on the Catalogue Raisonné that has been published.

## News Highlights

- **Exhibition: "Arshile Gorky: New York City"**
  - **Location**: Hauser & Wirth, New York
  - **Dates**: September 4 – October 26, 2024

- **Catalogue Raisonné Release**
  - The Foundation published the *Arshile Gorky Catalogue Raisonné* on November 12, 2021.

- **Exhibition: "Arshile Gorky: Beyond The Limit"**
  - **Location**: Hauser & Wirth, New York
  - **Dates**: November 16 – December 23, 2021.

The website showcases Gorky's significance in the art world, especially through upcoming and past exhibitions.

# Let's try more websites

Note that this will only work on websites that can be scraped using this simplistic approach.

Websites that are rendered with Javascript, like React apps, won't show up. Also Websites protected with CloudFront (and similar) may give 403 errors.

But many websites will work just fine!

In [15]:
display_summary("https://netflix.com")

# Summary of Netflix Germany Website

The **Netflix Germany** website offers a subscription service that provides access to an extensive library of movies, TV shows, anime, documentaries, and award-winning Netflix originals. The service starts at **€4.99** and can be canceled at any time. 

## Key Features
- **Affordable Plans**: Subscriptions range from €4.99 to €19.99 per month, with no contracts or extra costs.
- **Watch Anywhere**: Content can be streamed on various devices, including smart TVs, tablets, and smartphones. 
- **Offline Viewing**: Users can download shows and movies to watch without an internet connection.
- **Kids’ Profiles**: A dedicated space for children that includes parental controls and family-friendly content.
- **Trending Content**: Users can see what is currently popular, with highlights including films like *Meet Me Next Christmas* and *Alita: Battle Angel*.

## Conclusion
Netflix Germany provides an attractive streaming solution for viewers looking for a wide range of entertainment options at a reasonable price, with added features for family use and flexibility in subscription management.

## Conclusion

Here we explored how to interact with the Cloud API of a leading AI model like OpenAI's Frontier Model. We specifically applied the power of AI to Summarization — a fundamental use case in Generative AI. Summarization has countless business applications, such as:

- Summarizing news articles for quick updates.
- Condensing financial performance reports for stakeholders.
- Crafting concise resumes or cover letters from extensive details.

The possibilities are endless.